# Installation ⚡

In [ ]:
! pip install -U RelevanceAI
! pip install transformers
! pip install sentence_transformers

# Setup 🛠

## Client 🤖

In [ ]:
from relevanceai import Client

client = Client()

## Variables 🗒

In [4]:
input_field = 'product_title'               # Field to process for emotion
output_field = f"{input_field}_emotion"     # Name of the field to save the emotion results
dataset_id = 'emotion-experience'           # Name of the dataset to upload the data to

model_name = "joeddav/distilbert-base-uncased-go-emotions-student"  #  any transformer model trained for emotion recognition

# Emotion Class 🙂😐😠

In [9]:
from typing import List
from transformers import pipeline

class Emotion:
  def __init__(self, model_name:str = "joeddav/distilbert-base-uncased-go-emotions-student"):
    self.emotion = pipeline('sentiment-analysis', model_name)

  def add_emotion_field(self, docs:List, input_field:str, output_field:str):
    idx = []
    text_data = []
    for i, doc in enumerate(docs):
      if input_field in doc and doc[input_field]:
        idx.append(i)
        text_data.append(doc[input_field])
    emotion_res = self.emotion(text_data)

    for i, ix in enumerate(idx):
      docs[ix][output_field] = emotion_res[i]['label']

    return docs

In [10]:
em = Emotion(model_name)

# Data 📚

In [3]:
from relevanceai.utils.datasets import get_ecommerce_dataset_clean

docs = get_ecommerce_dataset_clean()
len(docs)

739

# Add emotion field to data ✨

In [12]:
docs = em.add_emotion_field(docs, input_field, output_field)

In [13]:
docs[5]

{'_id': '711158777',
 'product_image': 'https://thumbs3.ebaystatic.com/d/l225/m/mNxuXg3264NnGa85d_NqExQ.jpg',
 'product_link': 'https://www.ebay.com/itm/1-Victorias-Secret-Vanilla-Lace-Scented-Massage-Oil-Sparkling-Caress-Rare-HTF-/310992687930?pt=LH_DefaultDomain_0&hash=item48689bd33a',
 'product_price': '$24.59 ',
 'product_title': "1 Victoria's Secret Vanilla Lace Scented Massage Oil Sparkling Caress Rare HTF",
 'product_title_emotion': 'surprise',
 'query': 'Vanilla Scented Perfumes',
 'source': 'eBay'}

# Upload data 📫

In [ ]:
ds = client.Dataset(dataset_id)
ds.insert_documents(docs)